In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import os
from PIL import Image
import rasterio
import time
from graph_tool.all import *

In [ ]:
from power_planner.utils.utils import angle, plot_path, shift_surface

## Test graph operations

In [ ]:
g_test = Graph()
g_test.add_vertex(20)
weight = g_test.new_edge_property("float")
edges = []
for i in range(3,16):
    edges.append([i,i+1, i])
g_test.add_edge_list(edges, eprops=[weight])

In [ ]:
edges = [[2,3,1], [16, 17,1]]
g_test.add_edge_list(edges)

In [ ]:
v_path, e_path = shortest_path(g_test, g_test.vertex(2), g_test.vertex(17), weights=weight)

In [ ]:
for e in e_path:
    print(weight[e])

### Load everything

In [ ]:
base_path = "../../outputs/"
graph_name = "path_01128"

In [ ]:
g = load_graph(base_path+graph_name+"_graph.xml.gz")

In [ ]:
w = g.ep.weight

In [ ]:
costs = np.load(base_path+"cost_rest.npy")

In [ ]:
pos2node = np.load(base_path+graph_name+"_pos2node.npy")

In [ ]:
node_pos = []
for i in range(n_vertices):
    pos_x, pos_y = np.where(pos2node==i)
    node_pos.append((pos_x[0], pos_y[0]))

In [ ]:
n_vertices = len(list(g.vertices()))
n_edges = len(list(g.edges()))
print("size of graph:", n_vertices, n_edges)

### Tests for correctness

In [ ]:
plt.imshow(pos2node)
plt.colorbar()
plt.show()

In [ ]:
plt.imshow(costs)
plt.show()

In [ ]:
v1 = g.vertex(pos2node[40,20])

In [ ]:
for n in v1.out_neighbors():
    v_ind = g.vertex_index[n]
    pos_x, pos_y = np.where(pos2node==v_ind)
    print(pos_x[0], pos_y[0])
    print(w[g.edge(v1, v_ind)])

### Define line graph and helper arrays

In [ ]:
g_line = Graph()

In [ ]:
_ = g_line.add_vertex(n_edges)

In [ ]:
# not necessary!
edge_mapping = []
for e in g.edges():
    edge_mapping.append(tuple(e))

In [ ]:
max_shape = (int(np.max(pos2node))+1, int(np.max(pos2node))+1)

In [ ]:
edge_to_node = np.ones(max_shape)
edge_to_node *= -1
for k, (i,j) in enumerate(edge_mapping):
    edge_to_node[int(i),int(j)] = k

In [ ]:
# plt.figure(figsize=(20,10))
plt.imshow(costs)
plt.show()

### Fill line graph with edges

In [ ]:
len(edges)

In [ ]:
counter = 10
edges = []
for i, v in enumerate(g.vertices()):
    for in_nb in v.in_neighbours():
        for out_nb in v.out_neighbours():
            in_nb_ind = node_pos[int(in_nb)]
            out_nb_ind = node_pos[int(out_nb)]
            pos = node_pos[i]
            # vector between: subtract two pos tuples
            vec1 = np.subtract(in_nb_ind, pos)
            vec2 = np.subtract(pos, out_nb_pos)
            angle_cost = angle(vec1, vec2)/(0.5*np.pi)
            # print(node_pos[int(in_nb)], node_pos[g.vertex_index[v]], node_pos[int(out_nb)])
            # print(angle(vec1, vec2))
            if angle_cost <= 1:
                v1_line = edge_to_node[int(in_nb), i]
                v2_line = edge_to_node[i, int(out_nb)]
                # w[g.edge(v1, v_ind)]
                cost_before = costs[pos[0], pos[1]]
                # print(cost_before, angle_cost)
                edges.append([v1_line, v2_line, 0.5 * angle_cost + cost_before])
    #if i>counter:
     #   break

In [ ]:
w_line = g_line.new_edge_property("float")
g_line.add_edge_list(edges,  eprops=[w_line])

In [ ]:
len(list(g_line.edges()))

## Shortest path old

In [ ]:
source = pos2node[40,6]
dest = pos2node[23,60]

In [ ]:
vertices_path, _ = shortest_path(
                g,
                g.vertex(source),
                g.vertex(dest),
                weights=w,
                negative_weights=True
            )
path = [node_pos[g.vertex_index[v]] for v in vertices_path]

In [ ]:
list(g.vertex(0).out_edges())

In [ ]:
plot_path(costs, path, buffer=0)

## Shortest path new

In [ ]:
source_line = g_line.add_vertex()
dest_line = g_line.add_vertex()

In [ ]:
source_dest_edges = []
for e_out in g.vertex(source).out_edges():
    e_out = tuple(e_out)
    node_line = edge_to_node[int(e_out[0]), int(e_out[1])]
    print(node_line)
    source_dest_edges.append([g_line.vertex_index[source_line], node_line, 0])
    
for e_out in g.vertex(dest).in_edges():
    e_out = tuple(e_out)
    node_line = edge_to_node[int(e_out[0]), int(e_out[1])]
    print(node_line)
    source_dest_edges.append([node_line, g_line.vertex_index[dest_line], 0])

In [ ]:
g_line.add_edge_list(source_dest_edges,  eprops=[w_line])

In [ ]:
vertices_path, _ = shortest_path(
                g_line,
                source_line,
                dest_line,
                weights=w_line,
                negative_weights=True
            )
# path = [node_pos[g.vertex_index[v]] for v in vertices_path]

In [ ]:
path_line = []
for i, v in enumerate(vertices_path[1:-1]):
    v_ind_line = g_line.vertex_index[v]
    edge_actual = tuple(list(g.edges())[v_ind_line])
    if i==0:
        path_line.append(node_pos[g.vertex_index[edge_actual[0]]])
    path_line.append(node_pos[g.vertex_index[edge_actual[1]]])

In [ ]:
list(g.edges())[0]

In [ ]:
plot_path(costs, path_line, buffer=0)

In [ ]:
plot_path(costs, path_line, buffer=0)

In [ ]:
p = # to be filled
plt.figure(figsize=(20,10))
for i, f in enumerate(["cost_only", "angle_only", "angle+cost"]):
    img = plt.imread(p+f+".png")
    plt.subplot(1,3,i+1)
    plt.imshow(img)
    plt.axis("off")
    plt.title(f)
plt.tight_layout()
plt.savefig("path_comparison.png")
plt.show()

## New donut tuples

In [ ]:
def get_donut(radius_low, radius_high):
    """
    Compute all indices of points in donut around (0,0)
    :param radius_low: minimum radius
    :param radius_high: maximum radius
    :returns: tuples of indices of points with radius between radius_low 
    and radius_high around (0, 0)
    """
    img_size = int(radius_high + 10)
    # xx and yy are 200x200 tables containing the x and y coordinates as values
    # mgrid is a mesh creation helper
    xx, yy = np.mgrid[-img_size:img_size, -img_size:img_size]
    # circle equation
    circle = (xx)**2 + (yy)**2
    # donuts contains 1's and 0's organized in a donut shape
    # you apply 2 thresholds on circle to define the shape
    donut = np.logical_and(
        circle <= (radius_high**2), circle >= (radius_low**2)
    )
    pos_x, pos_y = np.where(donut > 0)
    return pos_x - img_size, pos_y - img_size

def get_half_donut(radius_low, radius_high, vec, angle_max=0.5 * np.pi):
    """
    Returns only the points with x >= 0 of the donut points (see above)
    :param radius_low: minimum radius
    :param radius_high: maximum radius
    :returns: tuples of indices of points with radius between radius_low 
    and radius_high around (0, 0)
    """
    pos_x, pos_y = get_donut(radius_low, radius_high)
    new_tuples = []
    shift_vals = []
    for i, j in zip(pos_x, pos_y):
        # if i > 0 or i == 0 and j > 0:
        # if i * vec[0] + j * vec[1] >= 0:
        ang = angle([i, j], vec)
        if ang <= angle_max:
            new_tuples.append((i, j))
            shift_vals.append(ang + 0.1)
    return new_tuples, shift_vals

In [ ]:
donut = get_donut(2.5, 5)

In [ ]:
vec = [1,1]
tuple_zip = list(zip(donut[0], donut[1]))
min_angle = 3*np.pi/4

In [ ]:
def get_lg_donut(radius_low, radius_high, vec, min_angle= 3*np.pi/4):
    donut = get_donut(radius_low, radius_high)
    tuple_zip = list(zip(donut[0], donut[1]))
    linegraph_tuples = []
    for (i,j) in tuple_zip:
        # if in incoming half
        if i*vec[0] + j* vec[1] <= 0:
            for (k,l) in tuple_zip:
                ang = angle([k,l], [i,j])
                # min angle and general outgoing edges half
                if ang >= min_angle and k*vec[0] + l* vec[1] >= 0:
                    print(ang)
                    linegraph_tuples.append([[i,j], [k,l], round(1-(ang/np.pi),2)])
    return linegraph_tuples

In [ ]:
lg_tuples = get_lg_donut(0.75,1.5, [1,1])

In [ ]:
lg_tuples

In [ ]:
example = lg_tuples[4]

In [ ]:
example

In [ ]:
np.where(np.roll(costs, example[1], axis=(0,1)))

In [ ]:
np.where(costs)

### Shift in both directions

In [ ]:
example_test = [[-1,-1],[1,1], 2]
costs_test = np.zeros((3,3))
costs_test[1,1]=1
costs_test[2,2]=1
costs_test[0,0]=1

In [ ]:
in_edges = shift_surface(costs_test, np.asarray(example_test[0])*(-1))
out_edges = shift_surface(costs_test, np.asarray(example_test[1]))
possible_in_edges = np.where(costs_test * in_edges)
possible_out_edges = np.where(costs_test * out_edges)
print(possible_in_edges, possible_in_edges)
in_edge_2 = np.swapaxes(np.vstack(possible_in_edges), 1,0)
in_edge_1 = in_edge_2 + np.array(example_test[0])
node_to_edge = np.concatenate([np.expand_dims(in_edge_1, 1), np.expand_dims(in_edge_2, 1)], axis=1)

In [ ]:
costs_bin = costs_test>0
out = valid_edges(costs_bin, example)

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(out) #out_node.astype(int)+costs_bin.astype(int))
plt.show()

In [ ]:
np.where(in_node)

### Add nodes and edges

In [ ]:
mask = costs>0

### First attempt

In [ ]:
# for shift in shift tuples:
shift = example
in_edges = shift_surface(mask, np.asarray(shift[0])*(-1))
possible_in_edges = np.where(mask * in_edges)
in_edge_2 = np.swapaxes(np.vstack(possible_in_edges), 1,0)
in_edge_1 = in_edge_2 + np.array(shift[0])
in_edge = np.concatenate([np.expand_dims(in_edge_1, 1), np.expand_dims(in_edge_2, 1)], axis=1)

out_edges = shift_surface(mask, np.asarray(shift[1])*(-1))
possible_out_edges = np.where(mask * out_edges)
out_edge_2 = np.swapaxes(np.vstack(possible_out_edges), 1,0)
out_edge_1 = out_edge_2 + np.array(shift[1])
out_edge = np.concatenate([np.expand_dims(out_edge_1, 1), np.expand_dims(out_edge_2, 1)], axis=1)

### Working version: with edge mapping dictionary

In [ ]:
shifts, _ = get_half_donut(0.75, 1.5, [-17, 54])

In [ ]:
# Build edge dictionary
edge_array = []
for i in range(len(shifts)):
    out_edges = shift_surface(mask, np.asarray(shifts[i])*(-1))
    possible_out_edges = np.where(mask * out_edges)
    out_edge_2 = np.swapaxes(np.vstack(possible_out_edges), 1,0)
    out_edge_1 = out_edge_2 + np.array(shifts[i])
    out_edge = np.concatenate([np.expand_dims(out_edge_2, 1), np.expand_dims(out_edge_1, 1)], axis=1)
    edge_array.append(out_edge)
    
edge_lists_concat = np.concatenate(edge_array, axis=0)
edge_dict = {}
edge_dict = {(tuple(edge_lists_concat[i, 0]), tuple(edge_lists_concat[i,1])):i for i in range(len(edge_lists_concat))}

In [ ]:
def valid_edges(mask, shift):
    in_node = shift_surface(mask, np.asarray(shift[0])*(-1))
    out_node = shift_surface(mask, np.asarray(shift[1])*(-1))
    stacked = np.asarray([mask, in_node, out_node])
    return np.all(stacked, axis =0)

In [ ]:
lg_tuples = get_lg_donut(0.75, 1.5, [-17, 54]) # different before!

In [ ]:
graph = Graph()
_ = graph.add_vertex(len(edge_lists_concat))
weight = graph.new_edge_property("float")

In [ ]:
# for every angle in the new angle tuples
for shift in lg_tuples:
    print(shift[2])
    all_angles = valid_edges(mask,shift)
    node_inds = np.swapaxes(np.vstack(np.where(all_angles)), 1,0)
    node_cost = costs[all_angles]
    in_node = node_inds + shift[0]
    out_node = node_inds + shift[1]
    edges_lg = []
    for i in range(len(node_inds)):
        e1 = edge_dict[(tuple(in_node[i]), tuple(node_inds[i]))]
        e2 = edge_dict[(tuple(node_inds[i]), tuple(out_node[i]))]
        edges_lg.append([e1, e2, shift[2]]) # node_cost[i]])
    graph.add_edge_list(edges_lg, eprops=[weight])

In [ ]:
lg_tuples

### Test 

In [ ]:
edges_lg[20]

In [ ]:
edge_lists_concat[34]

In [ ]:
edge_lists_concat[972]

In [ ]:
len(edges_lg)*len(lg_tuples)

### Add start and end

In [ ]:
source = [40, 6]
dest = [23, 60]


possible_start_edges = []
for shift in shifts:
    neighbor = np.asarray(source) + np.asarray(shift)
    node_val = edge_dict.get(
        (tuple(source), tuple(neighbor)), -1
    )
    if node_val > 0:
        possible_start_edges.append(node_val)

possible_dest_edges = []
for shift in shifts:
    neighbor = np.asarray(dest) - np.asarray(shift)
    node_val = edge_dict.get(
        (tuple(neighbor), tuple(dest)), -1
    )
    if node_val > 0:
        possible_dest_edges.append(node_val)

start_v = graph.add_vertex()
dest_v = graph.add_vertex()
start_ind = graph.vertex_index[start_v]
dest_ind = graph.vertex_index[dest_v]

start_edges = [[start_ind, u, 1] for u in possible_start_edges]
dest_edges = [[u, dest_ind, 1] for u in possible_dest_edges]
graph.add_edge_list(start_edges)
graph.add_edge_list(dest_edges)

print([start_v, dest_v])


In [ ]:
vertices_path, _ = shortest_path(
                graph,
                start_v,
                dest_v,
                weights=weight,
                negative_weights=True
            )
# path = [node_pos[g.vertex_index[v]] for v in vertices_path]

In [ ]:
edge_mapping = [k for k, _ in sorted(edge_dict.items(), key=lambda item: item[1])]

In [ ]:
out_path = [edge_mapping[graph.vertex_index[v]][0] for v in vertices_path[1:-1]]
out_path.append(edge_mapping[graph.vertex_index[vertices_path[-2]]][1])

In [ ]:
plot_path(costs, out_path, buffer=0)

## Compute sizes

len(edges_lg)

In [ ]:
len(lg_tuples)

In [ ]:
nodes_orig = len(list(g.vertices()))
edges_orig = len(list(g.edges()))
donuts = get_half_donut(0.75, 1.5, [1,1])

print("nodes", nodes_orig, "neighrbors", len(donuts[0]), "edges", edges_orig)

In [ ]:
nodes_lg = len(list(graph.vertices()))
edges_lg = len(list(graph.edges()))
# donuts = get_half_donut(0.75, 1.5, [1,1])
print("nodes lg", nodes_lg, "edges lg", edges_lg)


In [ ]:
print(len(start_edges), len(dest_edges))

In [ ]:
lg_donut_new = get_lg_donut(0.75, 1.5, [1,1]) # different before!

In [ ]:
first = np.asarray([l[0] for l in lg_donut_new])
second = np.asarray([l[1] for l in lg_donut_new])

In [ ]:
plt.scatter(first[:,0], first[:,1])
plt.scatter(second[:,0], second[:,1])

--> only so much bigger in notebook because more lg tuples taken into account